In [42]:
import os

from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "2wsx3edc"

graph = Neo4jGraph()

In [43]:
graph.query(
    """
MATCH (n:`__Community__`)<-[:IN_COMMUNITY]-()<-[:HAS_ENTITY]-(c)
WITH n, count(distinct c) AS chunkCount
SET n.weight = chunkCount"""
)

[]

In [44]:
from langchain_openai import AzureOpenAIEmbeddings

embedding = AzureOpenAIEmbeddings(
    model="text-embedding-ada-002",
    azure_endpoint='https://lang-chain-dev.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-05-15',
    azure_deployment='text-embedding-ada-002',
    openai_api_version='2023-05-15'
)

In [45]:
from langchain_community.vectorstores import Neo4jVector

lc_retrieval_query = """
WITH collect(node) as nodes
// Entity - Text Unit Mapping
WITH
collect {
    UNWIND nodes as n
    MATCH (n)<-[:HAS_ENTITY]->(c:__Chunk__)
    WITH c, count(distinct n) as freq
    RETURN c.content AS chunkText
    ORDER BY freq DESC
    LIMIT $topChunks
} AS text_mapping,
// Entity - Report Mapping
collect {
    UNWIND nodes as n
    MATCH (n)-[:IN_COMMUNITY]->(c:__Community__)
    WITH c, c.rank as rank, c.weight AS weight
    RETURN c.summary 
    ORDER BY rank, weight DESC
    LIMIT $topCommunities
} AS report_mapping,
// Outside Relationships 
collect {
    UNWIND nodes as n
    MATCH (n)-[r:RELATED]-(m) 
    WHERE NOT m IN nodes
    RETURN r.description AS descriptionText
    ORDER BY r.rank, r.weight DESC 
    LIMIT $topOutsideRels
} as outsideRels,
// Inside Relationships 
collect {
    UNWIND nodes as n
    MATCH (n)-[r:RELATED]-(m) 
    WHERE m IN nodes
    RETURN r.description AS descriptionText
    ORDER BY r.rank, r.weight DESC 
    LIMIT $topInsideRels
} as insideRels,
// Entities description
collect {
    UNWIND nodes as n
    RETURN n.description AS descriptionText
} as entities
// We don't have covariates or claims here
RETURN {Chunks: text_mapping, Reports: report_mapping, 
       Relationships: outsideRels + insideRels, 
       Entities: entities} AS text, 1.0 AS score, {} AS metadata
"""

vectorstore = Neo4jVector.from_existing_graph(embedding=embedding, 
                                    index_name="description_embedding",
                                    node_label='__Entity__', 
                                    embedding_node_property='description_embedding', 
                                    text_node_properties=['description'],
                                    retrieval_query=lc_retrieval_query)

In [46]:
topChunks = 3
topCommunities = 3
topOutsideRels = 10
topInsideRels = 10
topEntities = 10

In [47]:
os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "2wsx3edc"

In [48]:
docs = vectorstore.similarity_search(
    "你對台灣人壽知道多少",
    k=topEntities,
    params={
        "topChunks": topChunks,
        "topCommunities": topCommunities,
        "topOutsideRels": topOutsideRels,
        "topInsideRels": topInsideRels,
    },
)
print(docs[0].page_content)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: RELATED)} {position: line: 25, column: 18, offset: 679} for query: "CALL db.index.vector.queryNodes($index, $k, $embedding) YIELD node, score \nWITH collect(node) as nodes\n// Entity - Text Unit Mapping\nWITH\ncollect {\n    UNWIND nodes as n\n    MATCH (n)<-[:HAS_ENTITY]->(c:__Chunk__)\n    WITH c, count(distinct n) as freq\n    RETURN c.content AS chunkText\n    ORDER BY freq DESC\n    LIMIT $topChunks\n} AS text_mapping,\n// Entity - Report Mapping\ncollect {\n    UNWIND nodes as n\n    MATCH (n)-[:IN_COMMUNITY]->(c:__Community__)\

Entities:
- 限以購買增額繳清保險金額辦理
Reports:
- 在這個社區中，有多個節點和關係。

節點包括：
- 保險金受益人 (Person)
- 宣告利率 (Concept)
- 儲存生息 (Financial_strategy)
- 要保人 (Person)
- 本公司 (Organization)
- 增值回饋分享金 (Financial_product)，描述為限以購買增額繳清保險金額辦理

關係包括：
- 增值回饋分享金會帶來利益給要保人和保險金受益人
- 增值回饋分享金由本公司提供並計算宣告利率
- 要保人可以和本公司終止合同
- 本公司負責處理增值回饋分享金並將其返還給要保人
- 儲存生息適用於增值回饋分享金並帶來利益給要保人
Chunks:
- ，依本契約約定 以現金給付增值回饋分享金予要保人，惟須於第 六保單年度屆滿後之每一保單週年日起，始得依本款方式給付。   三、儲存生息：選擇儲存生息者，各年度之增值回饋分享金將按各保單週年日當月之宣告利率依 據年複利方式，累積至要保人請求時給付，或至被保險人身故或本契約終止時，由本公司主動一 併給付。但在本公司給付受益人保險金而終止契約的情形，要保人未請求之增值回饋分享金及其 孳息，由該保險金受益人受領。惟須於第六保單年度屆滿後之每一保單週年日起，始得依本款方 式給付。   要保人若未選擇者，則視為選擇購買增額繳清保險金額，並得於本契約有效 期間內，以書面通知 本公司變更前項給付方式
- ，以書面通知 本公司變更前項給付方式，惟第六保單年度屆滿前，增值回饋分享金限以購買增額繳清保險金額 辦理。  被保險人保險年齡到達 16歲前，其增值回饋分享金於繳費期間應採抵繳保險費之方式辦理。但 因繳費期間已屆滿而無法抵繳保險費者，本公司改按各保單週年日當月之宣告利率，以年複利方 式儲存生息至被保險人保險年齡到達 16歲時，以累計儲存生息之金額一次計算增額繳清保險金 額，其後保單年度適用第一項規定。   要保人終止本契約，或被保險人於保險年齡達 16歲前死亡者，本公司應退還歷年累計儲存生息 之金額予要保人。   本公司於每一保 單年度屆滿後，應將該增值回饋分享金之金額
- 第 1 頁，共  4 頁 台灣人壽 美鑫美利美元利率變動型終身壽險   保險商品內容說明   【承保範圍】   【增值回饋分享金的給付及

In [49]:
docs

[Document(page_content='Entities:\n- 限以購買增額繳清保險金額辦理\nReports:\n- 在這個社區中，有多個節點和關係。\n\n節點包括：\n- 保險金受益人 (Person)\n- 宣告利率 (Concept)\n- 儲存生息 (Financial_strategy)\n- 要保人 (Person)\n- 本公司 (Organization)\n- 增值回饋分享金 (Financial_product)，描述為限以購買增額繳清保險金額辦理\n\n關係包括：\n- 增值回饋分享金會帶來利益給要保人和保險金受益人\n- 增值回饋分享金由本公司提供並計算宣告利率\n- 要保人可以和本公司終止合同\n- 本公司負責處理增值回饋分享金並將其返還給要保人\n- 儲存生息適用於增值回饋分享金並帶來利益給要保人\nChunks:\n- ，依本契約約定 以現金給付增值回饋分享金予要保人，惟須於第 六保單年度屆滿後之每一保單週年日起，始得依本款方式給付。   三、儲存生息：選擇儲存生息者，各年度之增值回饋分享金將按各保單週年日當月之宣告利率依 據年複利方式，累積至要保人請求時給付，或至被保險人身故或本契約終止時，由本公司主動一 併給付。但在本公司給付受益人保險金而終止契約的情形，要保人未請求之增值回饋分享金及其 孳息，由該保險金受益人受領。惟須於第六保單年度屆滿後之每一保單週年日起，始得依本款方 式給付。   要保人若未選擇者，則視為選擇購買增額繳清保險金額，並得於本契約有效 期間內，以書面通知 本公司變更前項給付方式\n- ，以書面通知 本公司變更前項給付方式，惟第六保單年度屆滿前，增值回饋分享金限以購買增額繳清保險金額 辦理。  被保險人保險年齡到達 16歲前，其增值回饋分享金於繳費期間應採抵繳保險費之方式辦理。但 因繳費期間已屆滿而無法抵繳保險費者，本公司改按各保單週年日當月之宣告利率，以年複利方 式儲存生息至被保險人保險年齡到達 16歲時，以累計儲存生息之金額一次計算增額繳清保險金 額，其後保單年度適用第一項規定。   要保人終止本契約，或被保險人於保險年齡達 16歲前死亡者，本公司應退還歷年累計儲存生息 之金額予要保人。   本公司於每一保 單年度屆滿後，應將該增值回饋分享金之金額\n- 第 1 頁，共  4 頁 台灣人壽 美鑫美利